# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [8]:
%load_ext autoreload
%autoreload 2
from helpers.irp_integration import IRPClient
import json, secrets, string
from helpers.irp_integration.utils import extract_analysis_id_from_workflow_response, get_workspace_root

irp_client = IRPClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Configurations

In [11]:
suffix = ''.join(secrets.choice(string.ascii_lowercase + string.digits) for _ in range(6))
server_name = "databridge-1"
edm_name = f"CBHU_Automated_{suffix}" # ensure unique within IRP
edm_data_version = "22"
portfolio_name = "CBHU_Automated" # ensure unique within the edm
accounts_file_name = "Modeling_202503_Moodys_CBHU_Account.csv"
locations_file_name = "Modeling_202503_Moodys_CBHU_Location.csv"
analysis_job_name = "CBHU Analysis"
analysis_profile_name = "DLM CBHU v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
event_rate_scheme_name = "RMS 2025 Stochastic Event Rates"

### Create EDM

In [12]:
database_servers = irp_client.edm.search_database_servers(filter=f"serverName=\"{server_name}\"")
database_server_id = database_servers[0]['serverId']
exposure_sets = irp_client.edm.search_exposure_sets(filter=f"exposureSetName={edm_name}")
if (len(exposure_sets) > 1):
    raise Exception(f"{len(exposure_sets)} exposure sets found with name {edm_name}, please use a unique name")
exposure_set_id = irp_client.edm.create_exposure_set(name=edm_name)
edms = irp_client.edm.search_edms(filter=f"exposureName IN (\"{edm_name}\")")
if (len(edms) > 1):
    raise Exception(f"{len(edms)} EDMs found with name {edm_name}, please use a unique name")
create_edm_job_id = irp_client.edm.submit_create_edm_job(
    exposure_set_id=exposure_set_id,
    edm_name=edm_name,
    server_id=database_server_id)
job_response = irp_client.client.poll_risk_data_job_to_completion(create_edm_job_id)
print(json.dumps(job_response, indent=2))

Polling risk data job ID 23039834
Job status: QUEUED; Percent complete: 0
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 10
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 30
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 30
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 30
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 30
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 30
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 30
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 30
Polling risk data job ID 23039834
Job status: RUNNING; Percent complete: 30
Polling risk data job ID 23039834
Job status: FINISHED; Percent complete: 100
{
  "jobId": "23039834",
  "userName": "Partner - PremiumIQ key",
  "status": "FINISHED",
  "submittedAt": "2025-10-31T05:35:45.088Z",
  "startedAt": "2025-10-31T05

In [ ]:
# print(json.dumps(irp_client.edm.create_edm(edm_name=edm_name, server_name="databridge-1"), indent=2))

### Create Portfolio

In [ ]:
portfolio_response = irp_client.portfolio.create_portfolio(edm_name=edm_name, portfolio_name=portfolio_name, portfolio_number=portfolio_name)
portfolio_id = portfolio_response['id']
print(json.dumps(portfolio_response, indent=2))

### MRI Import

In [ ]:
# Use get_workspace_root() to get correct path in both VS Code and JupyterLab
workspace_root = get_workspace_root()
working_files_dir = workspace_root / "workflows" / "_Tools" / "files" / "working_files"

import_response = irp_client.mri_import.import_from_files(
    edm_name=edm_name,
    portfolio_id=portfolio_id,
    accounts_file=accounts_file_name,
    locations_file=locations_file_name,
    mapping_file="mapping.json",
    working_dir=str(working_files_dir)
)

print(json.dumps(import_response, indent=2))

### Create Treaty

In [ ]:
# New simplified approach using create_treaty_from_names() convenience method
treaty_name = "CBHU_treaty"

treaty_response = irp_client.treaty.create_treaty_from_names(
    edm_name=edm_name,
    treaty_name=treaty_name,
    treaty_type_name="Working Excess",
    currency_name="US Dollar",
    attachment_basis_name="Losses Occurring",
    attachment_level_name="Location",
    risk_limit=3000000,
    occur_limit=9000000,
    attach_pt=2000000,
    pcnt_covered=100,
    pcnt_placed=95,
    pcnt_ri_share=100,
    pcnt_retent=100,
    premium=0,
    num_of_reinst=99,
    reinst_charge=0,
    aggregate_limit=0,
    aggregate_deductible=0,
    priority=1,
    effect_date="2025-10-15T17:49:10.637Z",
    expire_date="2026-10-15T17:49:10.637Z",
    auto_assign_lobs=True
)
treaty_id = treaty_response['id']

print(f"Treaty created successfully!")
print(f"Treaty ID: {treaty_id}")
print(f"LOBs assigned: {treaty_response.get('lobs_assigned', 'N/A')}")

### Upgrade EDM Version

In [ ]:
edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Expected 1 EDM with name {edm_name}, found {len(edms)}")
exposure_id = edms[0]['exposureId']
upgrade_job_id = irp_client.edm.submit_upgrade_edm_data_version_job(exposure_id, edm_data_version)
job_response = irp_client.client.poll_workflow_to_completion(upgrade_job_id)
print(json.dumps(job_response, indent=2))

Polling risk data job ID 23040394
Workflow status: PENDING; Percent complete: 0
Polling risk data job ID 23040394
Workflow status: PENDING; Percent complete: 0
Polling risk data job ID 23040394
Workflow status: RUNNING; Percent complete: 0
Polling risk data job ID 23040394
Workflow status: RUNNING; Percent complete: 100
Polling risk data job ID 23040394
Workflow status: FINISHED; Percent complete: 100
{
  "id": 23040394,
  "userName": "Partner - PremiumIQ key",
  "status": "FINISHED",
  "submitTime": "2025-10-31T05:44:13.718Z",
  "startTime": "2025-10-31T05:44:32Z",
  "endTime": "2025-10-31T05:44:44Z",
  "name": "CBHU_Automated_w4gjrf",
  "type": "EDM_DATA_UPGRADE",
  "progress": 100,
  "priority": "medium",
  "entitlement": "RI-RISKMODELER",
  "resourceGroupId": "ba672cd9-3384-4cb3-9882-bbeb5f8478c6",
  "jobs": [
    {
      "id": "a53fd77e-bab7-4ad9-8f37-5ba70b71f1ae",
      "taskId": 1,
      "workflowId": 23040394,
      "status": "Succeeded",
      "submitTime": "2025-10-31T05:44:

In [ ]:
# print(json.dumps(irp_client.edm.upgrade_edm_version(edm_name), indent=2))

### GeoHaz

In [ ]:
print(json.dumps(
    irp_client.portfolio.geohaz_portfolio(edm_name, 
                                          portfolio_id, 
                                          geocode=True, 
                                          hazard_eq=True, 
                                          hazard_ws=True), 
    indent=2
))

### Execute Single Analysis

In [ ]:
analysis_response = irp_client.analysis.execute_analysis(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name,
    [treaty_id]
)
print(json.dumps(analysis_response, indent=2))

### Execute Multiple Analyses

In [ ]:
analysis_profile_name = "DLM CBHU v23"
output_profile_name = "Patched Portfolio Level Only (EP, ELT, Stats)"
tag = "test_tag"

analysis_job_name1 = "CBHU 2023 Rates"
event_rate_scheme_name1 = "RMS 2023 Stochastic Event Rates"
workflow_id1 = irp_client.analysis.submit_analysis_job(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name1,
    [treaty_id],
    [tag]
)

analysis_job_name2 = "CBHU 2025 Rates"
event_rate_scheme_name2 = "RMS 2025 Stochastic Event Rates"
workflow_id2 = irp_client.analysis.submit_analysis_job(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name2,
    [treaty_id],
    [tag]
)

workflow_ids = [workflow_id1, workflow_id2]
response = irp_client.analysis.poll_analysis_job_batch(workflow_ids)
print(json.dumps(response, indent=2))
analysis_ids = []
for workflow in response['workflows']:
    analysis_ids.append(extract_analysis_id_from_workflow_response(workflow))

### Group Analyses

In [ ]:
group_name = "CBHU Group"
print(json.dumps(irp_client.analysis.create_analysis_group(analysis_ids, group_name), indent=2))

### Export Analyses to RDM

In [ ]:
rdm_name = "CBHU RDM"
export_response = irp_client.rdm.export_analyses_to_rdm(rdm_name, analysis_ids)
print(json.dumps(export_response, indent=2))

### Delete EDM

In [ ]:
# delete_response = irp_client.edm.delete_edm(edm_name)
# print(json.dumps(delete_response, indent=2))